In [ ]:
import pandas as pd #importing libraries
import requests
import json
import ast
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

#Data_fetchng
allrecords=[]
for i in range(1,122):
    url=f"https://world.openfoodfacts.org/api/v2/search?categories=chocolates&fields=code,product_name,brands,nutriments&page_size=100&page={i}"
    response=requests.get(url)
    data=response.json()
    allrecords.extend(data['products'])

df = pd.DataFrame(allrecords)#converts into dataframe
df.to_csv("Correctrecords.csv",index=False)#saving the data as csv file
alldata=pd.read_csv('correctrecords.csv')#reads the csv file


mylist=alldata.to_dict(orient='records') #converts the dataframe into dictionary

for item in mylist: # To Convert string in 'nutriments' to a proper dictionary
    item['nutriments'] = ast.literal_eval(item['nutriments'])


df = pd.json_normalize(mylist)#normalizing the list

#creating the list to be extracted
required_columns= ['code', 
                   'product_name', 
                   'brands', 
                   'nutriments.energy-kcal_value',
                   'nutriments.energy-kj_value',
                   'nutriments.carbohydrates_value',
                   'nutriments.sugars_value',
                   'nutriments.fat_value',
                   'nutriments.saturated-fat_value',
                   'nutriments.proteins_value',
                   'nutriments.fiber_value',
                   'nutriments.salt_value',
                   'nutriments.sodium_value',
                   'nutriments.nova-group',
                   'nutriments.nutrition-score-fr',
                   'nutriments.fruits-vegetables-nuts-estimate-from-ingredients_100g'
                  ]

df=df[required_columns] #extracting only the required columns

#renaming the columns
df=df.rename(columns={'code':'product_code', 
                   'product_name':'product_name', 
                   'brands':'brand', 
                   'nutriments.energy-kcal_value':'energy-kcal_value',
                   'nutriments.energy-kj_value':'energy-kj_value',
                   'nutriments.carbohydrates_value':'carbohydrates_value',
                   'nutriments.sugars_value':'sugars_value',
                   'nutriments.fat_value':'fat_value',
                   'nutriments.saturated-fat_value':'saturated-fat_value',
                   'nutriments.proteins_value':'proteins_value',
                   'nutriments.fiber_value':'fiber_value',
                   'nutriments.salt_value':'salt_value',
                   'nutriments.sodium_value':'sodium_value',
                   'nutriments.nova-group':'nova-group',
                   'nutriments.nutrition-score-fr':'nutrition-score-fr',
                   'nutriments.fruits-vegetables-nuts-estimate-from-ingredients_100g':'fruits-vegetables-nuts-estimate-from-ingredients_100g'})

df.to_csv("Chocodata.csv",index=False)#saving the dataframe as a csv file

df.info()#checking the information of the data frame

df['product_name'].isnull()#checking if there is any null values in product_name column

df1=df.dropna(subset=['product_name'])#dropping the rows having null values
df1.info()#checking the information after dropping the null values

#replacing the 0's in the columns energy-kcal_value and energy-kj_value to nan values
df['energy-kcal_value'] = df['energy-kcal_value'].replace(0, np.nan)
df['energy-kj_value'] = df['energy-kj_value'].replace(0, np.nan)

#since both the below columns denote the same energy but in different units of measurement
#filling the nan values in the columns with the values in the other if present by converting its unit
df['energy-kcal_value'] = df['energy-kcal_value'].fillna(df['energy-kj_value'] / 4.184)
df['energy-kj_value'] = df['energy-kj_value'].fillna(df['energy-kcal_value'] * 4.184)

#creating a list of nutriments
nutriments = ['energy-kcal_value',
              'energy-kj_value',
              'carbohydrates_value',
              'sugars_value',
              'fat_value',
              'saturated-fat_value',
              'proteins_value',
              'fiber_value',
              'salt_value',
              'sodium_value',
              'nova-group',
              'nutrition-score-fr',
              'fruits-vegetables-nuts-estimate-from-ingredients_100g']
#using for loop to display the graphs of all items in the list to check if they are normally distributed
for i in nutriments:
    plt.figure(figsize=(6,4))
    sns.histplot(df[i], bins=2, kde=True)
    plt.title(f'Distribution of {i}')
    plt.xlabel(i)
    plt.ylabel('Frequency')
    plt.show()
#since none of them are normally distributed so nan values in the columns cannot be filled with mean values
#since all are numerical values getting the median values for all 
df[['energy-kcal_value',
              'energy-kj_value',
              'carbohydrates_value',
              'sugars_value',
              'fat_value',
              'saturated-fat_value',
              'proteins_value',
              'fiber_value',
              'salt_value',
              'sodium_value',
              'nova-group',
              'nutrition-score-fr',
              'fruits-vegetables-nuts-estimate-from-ingredients_100g']].median()

#creating a list
nutrients=['energy-kcal_value',
              'energy-kj_value',
              'carbohydrates_value',
              'sugars_value',
              'fat_value',
              'saturated-fat_value',
              'proteins_value',
              'fiber_value',
              'salt_value',
              'sodium_value',
              'nova-group',
              'nutrition-score-fr',
              'fruits-vegetables-nuts-estimate-from-ingredients_100g']

#creating a for loop to fill all the null values in all the columns by its median value
for i in nutrients:
    median_val = df[i].median()
    df[i].fillna(median_val, inplace=True)

df.isna().sum() #checking how many null values are there still

#since brand column is a text column it is not filled with median values. so it has to be filled with mode values
brandmode=df2['brand'].mode()

df['brand'].fillna(df['brand'].mode()[0], inplace=True) #filling the null values in the brand column with its mode value

df.isna().sum() #checking if null values are neutralized

#creating a new sugar to carbohydrates ratio column
df['sugar_to_carb_ratio'] = np.where(df['carbohydrates_value'] == 0, 0,df['sugars_value'] / df['carbohydrates_value'])

#creating a new calorie category column
calorie_categories = []
for kcal in df['energy-kcal_value']:
    if kcal < 100:
        calorie_categories.append('Low')
    elif 100 <= kcal < 200:
        calorie_categories.append('Moderate')
    else:
        calorie_categories.append('High')
df['calorie_category'] = calorie_categories

#creating a new category to classify sugar value
def classify_sugar(sugar):
    if sugar < 5:
        return 'Low sugar'
    elif 5 <= sugar < 15:
        return 'Moderate sugar'
    else:
        return 'High sugar'

df['sugar_category'] = df['sugars_value'].apply(classify_sugar)

#creating a new column mentioning if a product is ultra processed
def ultra_processed(group):
    if  group ==4:
        return 'yes'
    else:
        return 'No'

df['is_ultra_processed'] = df['nova-group'].apply(ultra_processed)

#finally saving the cleaned and organised data as a csv file
df.to_csv("Chocodatafinal.csv",index=False)
df=pd.read_csv("Chocodatafinal.csv") #reading the same data

#graphs
#Pie Chart – Proportion of Products in Each NOVA Group
plt.figure(figsize=(6,6))
nova_counts = df['nova-group'].value_counts()

plt.pie(
    nova_counts, 
    labels=nova_counts.index, 
    autopct='%1.1f%%', 
    startangle=140, 
    colors=sns.color_palette('pastel')
)
plt.title('Proportion of Products by NOVA Group')
plt.show()

#Histograms – Distribution of Calories, Sugars, or Sugar-to-Carb Ratio
numeric_cols = ['energy-kcal_value', 'sugars_value', 'sugar_to_carb_ratio']

for col in numeric_cols:
    plt.figure(figsize=(7,4))
    sns.histplot(df[col], bins=30, kde=True, color='teal')
    plt.title(f"Distribution of {col.replace('_', ' ').title()}")
    plt.xlabel(col)
    plt.ylabel('Frequency')
    plt.show()

#Box Plots – Spread and Outliers of Calories or Sugars Across Brands
top_brands = df['brand'].value_counts().nlargest(10).index
plt.figure(figsize=(10,6))
sns.boxplot(data=df[df['brand'].isin(top_brands)], x='brand', y='energy-kcal_value')
plt.title("Calories Across Top 10 Brands")
plt.xticks(rotation=90)
plt.xlabel("Brand")
plt.ylabel("Calories (kcal/100g)")
plt.show()

#Box plot- sugar value across top 10 brands
plt.figure(figsize=(10,6))
sns.boxplot(data=df[df['brand'].isin(top_brands)], x='brand', y='sugars_value', color='lightcoral')
plt.title("Sugar Levels Across Top 10 Brands")
plt.xticks(rotation=90)
plt.xlabel("Brand")
plt.ylabel("Sugars (g/100g)")
plt.show()

#Scatter Plot – Relationship Between Calories and Sugars
plt.figure(figsize=(6,5))
sns.scatterplot(data=df, x='energy-kcal_value', y='sugars_value', hue='calorie_category', palette='plasma', alpha=0.8)
plt.title("Calories vs Sugars")
plt.xlabel("Calories (kcal per 100g)")
plt.ylabel("Sugars (g per 100g)")
plt.show()

#Heatmap – Correlation Between Nutritional Values
nutrient_cols = [
    'energy-kcal_value', 'carbohydrates_value', 'sugars_value', 'fat_value',
    'proteins_value', 'fiber_value', 'salt_value', 'sodium_value', 'sugar_to_carb_ratio'
]

plt.figure(figsize=(10,6))
corr = df[nutrient_cols].corr()
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Correlation Heatmap of Nutritional Values")
plt.show()

#Top 10 Brands by Average Calories or Sugars
# Average calories
plt.figure(figsize=(7,5))
sns.scatterplot(
    data=df, x='energy-kcal_value', y='sugars_value',
    hue='calorie_category', palette='plasma', alpha=0.8
)
plt.title("Calories vs Sugars")
plt.xlabel("Calories (kcal/100g)")
plt.ylabel("Sugars (g/100g)")
plt.legend(title='Calorie Category', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

#Bar Charts – Number of Products per Calorie or Sugar Category
# Calorie category
cal_counts = df['calorie_category'].value_counts().sort_index()
colors3 = sns.color_palette('Set2', n_colors=len(cal_counts))

plt.figure(figsize=(6,4))
plt.bar(cal_counts.index, cal_counts.values, color=colors3)
plt.title("Number of Products by Calorie Category")
plt.xlabel("Calorie Category")
plt.ylabel("Number of Products")
plt.show()

# Sugar category
sug_counts = df['sugar_category'].value_counts().sort_index()
colors4 = sns.color_palette('Set3', n_colors=len(sug_counts))
plt.figure(figsize=(6,4))
plt.bar(sug_counts.index, sug_counts.values, color=colors4)
plt.title("Number of Products by Sugar Category")
plt.xlabel("Sugar Category")
plt.ylabel("Number of Products")
plt.show()

#creating tables in mysql local host through sqlalchemy

#importing libraries
from sqlalchemy import create_engine, text
from sqlalchemy.exc import OperationalError

#establishing connection between python and mysql
HOST = "localhost"
USER = "root"
PASSWORD = "root"
DATABASE_NAME = "chococrunch"

DB_URL = f"mysql+pymysql://{USER}:{PASSWORD}@{HOST}/{DATABASE_NAME}"

engine = create_engine(DB_URL)

#importing necessary libraries from sqlalchemy
from sqlalchemy import create_engine, MetaData, Table, Column, Text, ForeignKey, Float, Integer, String

metadata = MetaData()

#creating product_info table
product_info = Table(
    'product_info',
    metadata,
    Column('product_code', String(50), primary_key=True),  # length specified!
    Column('product_name', Text),
    Column('brand', Text)
)

#creating nutrient_info table
nutrient_info = Table(
    "nutrient_info",
    metadata,
    Column("product_code", String(50), ForeignKey("product_info.product_code"), primary_key=True),
    Column("energy-kcal_value", Float),
    Column("energy-kj_value", Float),
    Column("carbohydrates_value", Float),
    Column("sugars_value", Float),
    Column("fat_value", Float),
    Column("saturated-fat_value", Float),
    Column("proteins_value", Float),
    Column("fiber_value", Float),
    Column("salt_value", Float),
    Column("sodium_value", Float),
    Column("fruits-vegetables-nuts-estimate-from-ingredients_100g", Float),
    Column("nutrition-score-fr", Integer),
    Column("nova-group", Integer)
)

#creating derived_metrics table
derived_metrics = Table(
    'derived_metrics',
     metadata,
     Column("product_code", String(50), ForeignKey("product_info.product_code"), primary_key=True),
     Column("sugar_to_carb_ratio",Float),
     Column("calorie_category",Text),
     Column("sugar_category",Text),
     Column("is_ultra_processed",Text)
)
     
    


metadata.create_all(engine)

#preparing the list of data that has to be inserted into the respective table
product_columns = ['product_code', 'product_name', 'brand']
nutrient_columns = [
    'product_code',
    'energy-kcal_value',
    'energy-kj_value',
    'carbohydrates_value',
    'sugars_value',
    'fat_value',
    'saturated-fat_value',
    'proteins_value',
    'fiber_value',
    'salt_value',
    'sodium_value',
    'fruits-vegetables-nuts-estimate-from-ingredients_100g',
    'nutrition-score-fr',
    'nova-group'
    
]
derived_columns = ['product_code', 'sugar_to_carb_ratio', 'calorie_category', 'sugar_category', 'is_ultra_processed']

#converting the list of items into a dataframe dropping the duplicates
df_products = df[product_columns].drop_duplicates(subset='product_code')
df_nutrients = df[nutrient_columns].drop_duplicates(subset='product_code')
df_derived = df[derived_columns].drop_duplicates(subset='product_code')

#inserting the data into the tables
df_products.to_sql('product_info', con=engine, if_exists='append', index=False)
df_nutrients.to_sql('nutrient_info', con=engine, if_exists='append', index=False)
df_derived.to_sql('derived_metrics', con=engine, if_exists='append', index=False)

#for execution of sql queries using mysql, importing and establishing through mysql
import pymysql
mydb = pymysql.connect(
    host = "localhost",
    user = "root",
    password = "root",
    database = "chococrunch")

mycursor = mydb.cursor()

#Product_info_queries
#Count products per brand
query1="SELECT brand, COUNT(*) AS product_count FROM product_info GROUP BY brand;"
mycursor.execute(query1)
result=mycursor.fetchall()
df = pd.DataFrame(result,columns = [i[0] for i in mycursor.description])
df

#Count unique products per brand
query2="SELECT brand, COUNT(DISTINCT product_code) AS unique_products FROM product_info GROUP BY brand;"
mycursor.execute(query2)
result=mycursor.fetchall()
df = pd.DataFrame(result,columns = [i[0] for i in mycursor.description])
df

#Top 5 brands by product count
query3="SELECT brand, COUNT(*) AS product_count FROM product_info GROUP BY brand ORDER BY product_count DESC LIMIT 5;"
mycursor.execute(query3)
result=mycursor.fetchall()
df = pd.DataFrame(result,columns = [i[0] for i in mycursor.description])
df

#Products with missing product name
query4="SELECT * FROM product_info WHERE product_name IS NULL OR product_name = '';"
mycursor.execute(query4)
result=mycursor.fetchall()
df = pd.DataFrame(result,columns = [i[0] for i in mycursor.description])
df

#Number of unique brands
query5="SELECT COUNT(DISTINCT brand) AS unique_brands FROM product_info;"
mycursor.execute(query5)
result=mycursor.fetchall()
df = pd.DataFrame(result,columns = [i[0] for i in mycursor.description])
df

#Products with code starting with '3'
query6="SELECT * FROM product_info WHERE product_code LIKE '3%';"
mycursor.execute(query6)
result=mycursor.fetchall()
df = pd.DataFrame(result,columns = [i[0] for i in mycursor.description])
df

#queries for nutrient_info table
#Top 10 products with highest energy-kcal_value
query7="SELECT product_code, 'energy-kcal_value' FROM nutrient_info ORDER BY 'energy-kcal_value' DESC LIMIT 10;"
mycursor.execute(query7)
result=mycursor.fetchall()
df = pd.DataFrame(result,columns = [i[0] for i in mycursor.description])
df

#Average sugars_value per nova-group
query8="SELECT `nova-group`, AVG(`sugars_value`) AS avg_sugar FROM nutrient_info GROUP BY `nova-group`;"
mycursor.execute(query8)
result=mycursor.fetchall()
df = pd.DataFrame(result,columns = [i[0] for i in mycursor.description])
df

#Count products with fat_value > 20g
query9="SELECT COUNT(*) AS high_fat_products FROM nutrient_info WHERE fat_value > 20";
mycursor.execute(query9)
result=mycursor.fetchall()
df = pd.DataFrame(result,columns = [i[0] for i in mycursor.description])
df

#Average carbohydrates_value per product
query10="SELECT product_code, AVG(carbohydrates_value) AS avg_carbs FROM nutrient_info GROUP BY product_code;"
mycursor.execute(query10)
result=mycursor.fetchall()
df = pd.DataFrame(result,columns = [i[0] for i in mycursor.description])
df

#Products with sodium_value > 1g
query11= "SELECT product_code, sodium_value FROM nutrient_info WHERE sodium_value > 1;"
mycursor.execute(query11)
result=mycursor.fetchall()
df = pd.DataFrame(result,columns = [i[0] for i in mycursor.description])
df

#Count products with non-zero fruits-vegetables-nuts content
query12="SELECT COUNT(*) AS non_zero_fvn FROM nutrient_info WHERE `fruits-vegetables-nuts-estimate-from-ingredients_100g` > 0;"
mycursor.execute(query12)
result=mycursor.fetchall()
df = pd.DataFrame(result,columns = [i[0] for i in mycursor.description])
df

#Products with energy-kcal_value > 500
query13="SELECT product_code, `energy-kcal_value` FROM nutrient_info WHERE `energy-kcal_value` > 500;"
mycursor.execute(query13)
result=mycursor.fetchall()
df = pd.DataFrame(result,columns = [i[0] for i in mycursor.description])
df

#Count products per calorie_category
query14="SELECT calorie_category, COUNT(*) AS product_count FROM derived_metrics GROUP BY calorie_category;"
mycursor.execute(query14)
result=mycursor.fetchall()
df = pd.DataFrame(result,columns = [i[0] for i in mycursor.description])
df

#Count of High_sugar products
query15="SELECT COUNT(*) AS high_sugar_count FROM derived_metrics WHERE sugar_category = 'High sugar';"
mycursor.execute(query15)
result=mycursor.fetchall()
df = pd.DataFrame(result,columns = [i[0] for i in mycursor.description])
df

#Average sugar_to_carb_ratio for High Calorie products
query16="SELECT AVG(sugar_to_carb_ratio) AS avg_ratio_high_calorie FROM derived_metrics WHERE calorie_category = 'High';"
mycursor.execute(query16)
result=mycursor.fetchall()
df = pd.DataFrame(result,columns = [i[0] for i in mycursor.description])
df

#Products that are both High Calorie and High Sugar
query17="SELECT product_code, calorie_category, sugar_category FROM derived_metrics WHERE calorie_category = 'High' AND sugar_category = 'High sugar';"
mycursor.execute(query17)
result=mycursor.fetchall()
df = pd.DataFrame(result,columns = [i[0] for i in mycursor.description])
df

#Number of products marked as ultra-processed
query18="SELECT COUNT(*) AS ultra_processed_count FROM derived_metrics WHERE is_ultra_processed = 'yes';"
mycursor.execute(query18)
result=mycursor.fetchall()
df = pd.DataFrame(result,columns = [i[0] for i in mycursor.description])
df

#Products with sugar_to_carb_ratio > 0.7
query19="SELECT product_code, sugar_to_carb_ratio FROM derived_metrics WHERE sugar_to_carb_ratio > 0.7;"
mycursor.execute(query19)
result=mycursor.fetchall()
df = pd.DataFrame(result,columns = [i[0] for i in mycursor.description])
df

#Average sugar_to_carb_ratio per calorie_category
query20="SELECT calorie_category, AVG(sugar_to_carb_ratio) AS avg_ratio FROM derived_metrics GROUP BY calorie_category;"
mycursor.execute(query20)
result=mycursor.fetchall()
df = pd.DataFrame(result,columns = [i[0] for i in mycursor.description])
df

#Join queries
#Top 5 brands with most High Calorie products
query21="SELECT product_info.brand, COUNT(*) AS high_calorie_count FROM product_info JOIN derived_metrics ON product_info.product_code = derived_metrics.product_code WHERE derived_metrics.calorie_category = 'High' GROUP BY product_info.brand ORDER BY high_calorie_count DESC LIMIT 5;"
mycursor.execute(query21)
result=mycursor.fetchall()
df = pd.DataFrame(result,columns = [i[0] for i in mycursor.description])
df

#Average energy-kcal_value for each calorie_category
query22='''SELECT calorie_category, avg(`energy-kcal_value`) as avg_value
FROM derived_metrics
 JOIN nutrient_info
ON derived_metrics.product_code = nutrient_info.product_code
GROUP BY calorie_category;'''
mycursor.execute(query22)
result=mycursor.fetchall()
df = pd.DataFrame(result,columns = [i[0] for i in mycursor.description])
df

#Count of ultra-processed products
query23="SELECT p.brand, COUNT(*) AS ultra_processed_count FROM product_info p JOIN derived_metrics d ON p.product_code = d.product_code WHERE d.is_ultra_processed = 1 GROUP BY p.brand ORDER BY ultra_processed_count DESC;"
mycursor.execute(query23)
result=mycursor.fetchall()
df = pd.DataFrame(result,columns = [i[0] for i in mycursor.description])
df

#Products with High Sugar and High Calorie along with brand
query24="SELECT product_info.product_code, product_info.product_name, product_info.brand, derived_metrics.calorie_category, derived_metrics.sugar_category FROM product_info JOIN derived_metrics ON product_info.product_code = derived_metrics.product_code WHERE derived_metrics.calorie_category = 'High' AND derived_metrics.sugar_category = 'High sugar';"
mycursor.execute(query24)
result=mycursor.fetchall()
df = pd.DataFrame(result,columns = [i[0] for i in mycursor.description])
df

#Average sugar content per brand for ultra-processed products
query25="SELECT product_info.brand, AVG(nutrient_info.sugars_value) AS avg_sugar FROM product_info JOIN nutrient_info ON product_info.product_code = nutrient_info.product_code JOIN derived_metrics ON product_info.product_code = derived_metrics.product_code WHERE derived_metrics.is_ultra_processed = 'yes' GROUP BY product_info.brand ORDER BY avg_sugar DESC;"
mycursor.execute(query25)
result=mycursor.fetchall()
df = pd.DataFrame(result,columns = [i[0] for i in mycursor.description])
df

#Number of products with fruits/vegetables/nuts content in each calorie_category
query26="SELECT derived_metrics.calorie_category, COUNT(*) AS 'f/v/n_product_count' FROM derived_metrics JOIN nutrient_info ON derived_metrics.product_code = nutrient_info.product_code WHERE 'nutrient_info.fruits-vegetables-nuts-estimate-from-ingredients_100g' > 0 GROUP BY derived_metrics.calorie_category;"
mycursor.execute(query26)
result=mycursor.fetchall()
df = pd.DataFrame(result,columns = [i[0] for i in mycursor.description])
df

#Top 5 products by sugar_to_carb_ratio with their calorie and sugar category
query27="SELECT derived_metrics.product_code, derived_metrics.sugar_to_carb_ratio, derived_metrics.calorie_category, derived_metrics.sugar_category, product_info.brand FROM derived_metrics JOIN product_info ON derived_metrics.product_code = product_info.product_code ORDER BY 'derived_info.sugar_to_carb_ratio' DESC LIMIT 5;"
mycursor.execute(query27)
result=mycursor.fetchall()
df = pd.DataFrame(result,columns = [i[0] for i in mycursor.description])
df

